<img src="mioti.png" style="height: 100px">
<center style="color:#888">Data Science with Python</center>

# DSPy7 Workbook. Scrapy

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

Vamos a intentar extraer texto, autor y tags de las frase de la web http://quotes.toscrape.com/

**Inicialización**

In [1]:
import logging
import pandas as pd
import scrapy
from scrapy.crawler import CrawlerProcess
# Para ir viendo la salida de los comandos a medida que ocurre
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**Implementación del web scraper**

In [2]:
import json

class JsonWriterPipeline(object):
    """Indica qué hacer con los items que nos interesan (guardarlos como JSON lines)"""
    @classmethod
    def from_crawler(cls, crawler):
        settings = crawler.settings
        file_name = settings.get("FILE_NAME")
        return cls(file_name)
    
    def __init__(self, file_name):
        self.file_name = file_name

    def open_spider(self, spider):
        print(self.file_name)
        self.file = open(self.file_name, 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [9]:
class SimpleQuoteSpider(scrapy.Spider):
    """ Este es el web scraper propiamente dicho"""
    name = "simplebooks"
    start_urls = [
        'http://quotes.toscrape.com/page/1/',
    ]
    custom_settings = {
        'LOG_LEVEL': logging.DEBUG,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1},     
        'FILE_NAME': "quotes.jl"
    }
    
    def parse(self, response):
        """En este método indicamos cuáles son los elementos que nos interesan"""
        results = response.css('div.quote')
        page = response.css()
        print(page)
        for quote in results:
            yield{
                'text': quote.css('span.text::text').extract_first(),
                'author': quote.css('span small::text').extract_first(),
                'tags': quote.css('div.tags a.tag::text').extract(),
                'page': page
            }


**Ejecución del web scraper**

In [10]:
process = CrawlerProcess()
process.crawl(SimpleQuoteSpider)
process.start()

2020-04-15 17:13:47 [scrapy.utils.log] INFO: Scrapy 2.0.1 started (bot: scrapybot)
2020-04-15 17:13:47 [scrapy.utils.log] INFO: Versions: lxml 4.4.1.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.20.0, Twisted 20.3.0, Python 3.7.4 (default, Aug 13 2019, 15:17:50) - [Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.7, Platform Darwin-17.4.0-x86_64-i386-64bit
2020-04-15 17:13:47 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-04-15 17:13:47 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 10}
2020-04-15 17:13:47 [scrapy.extensions.telnet] INFO: Telnet Password: 7484333920774a89
2020-04-15 17:13:47 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2020-04-15 17:13:47 [scrapy.middleware] INFO: Enabled downloader mid

quotes.jl


ReactorNotRestartable: 

In [11]:
dfjson = pd.read_json('quotes.jl', lines=True)
dfjson

""
